<a href="https://colab.research.google.com/github/andreidore/aicrowd/blob/master/blitz_3/snake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm

In [ ]:
import os